# Connecting MSSQL Database to Databricks for analysis

In [0]:
%pip install dotenv

In [0]:
# importing important library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from dotenv import load_dotenv


## import for connection or bridge between Mssql and python

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, max, min, sum, avg, countDistinct


In [0]:
os.getcwd()

In [0]:
load_dotenv()

In [0]:
jdbc_host = os.getenv('host_env')
jdbc_port = os.getenv('port_env') 
jdbc_database = os.getenv('database_env') 
jdbc_user = os.getenv('user_env')
jdbc_password = os.getenv('password_env') 
print(os.getenv('user_env'))

In [0]:
jdbc_url = f"jdbc:sqlserver://{jdbc_host}:{jdbc_port};databaseName={jdbc_database}"


In [0]:
connection_properties = {
  "user": jdbc_user,
  "password": jdbc_password,
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


In [0]:

def load_data(query):
    df = spark.read.format("jdbc") \
        .option("url", jdbc_url) \
        .option("query", query)\
        .options(**connection_properties) \
        .load()
    return df

## importing all the data from database to variables

In [0]:
categories = spark.read.jdbc(url=jdbc_url, table="dbo.categories",properties=connection_properties)

customers = spark.read.jdbc(url=jdbc_url, table="dbo.customers", properties=connection_properties)

order_items = spark.read.jdbc(url=jdbc_url, table="dbo.order_items", properties=connection_properties)

orders = spark.read.jdbc(url=jdbc_url, table="dbo.orders", properties=connection_properties)

products = spark.read.jdbc(url=jdbc_url, table="dbo.products", properties=connection_properties)

staffs = spark.read.jdbc(url=jdbc_url, table="dbo.staffs", properties=connection_properties)

stocks = spark.read.jdbc(url=jdbc_url, table="dbo.stocks", properties=connection_properties)

stores = spark.read.jdbc(url=jdbc_url, table="dbo.stocks", properties=connection_properties)

brands = spark.read.jdbc(url=jdbc_url, table="dbo.brands", properties=connection_properties)

In [0]:
TotalSales="""
(
Select c.customer_id,
       c.first_name,
       c.last_name,
       o.order_id,
       o.order_date,
       SUM(oi.list_price*oi.quantity)*(1 - (oi.discount)) as TotalSales
from customers c
inner join 
orders o on c.customer_id = o.customer_id
inner join 
order_items oi on o.order_id = oi.order_id
GROUP BY c.customer_id, c.first_name, c.last_name
) as Customer_Sales
"""



In [0]:
query = """
SELECT
    c.customer_id,
    c.first_name,
    c.last_name,
    SUM((oi.list_price * oi.quantity) * (1 - oi.discount)) AS TotalSales
FROM dbo.customers AS c
JOIN dbo.orders AS o ON c.customer_id = o.customer_id
JOIN dbo.order_items AS oi ON o.order_id = oi.order_id
GROUP BY c.customer_id, c.first_name, c.last_name
"""

In [0]:
TotalSales_per_customer = load_data(query)

In [0]:
TotalSales_per_customer.sort("TotalSales", ascending=False).limit(10).display()

TotalSales_per_customer.write.jdbc(url=jdbc_url, table="TotalSales_per_customer", mode="overwrite", properties=connection_properties)